# Prediction of customers' travel pattern

# 1)-Importing key modules

In [1]:
import warnings
warnings.filterwarnings('ignore')
# For processing
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import scipy
import datetime as dt
from datetime import datetime
import seaborn as sns
plt.rcParams["figure.figsize"] = (16, 10)
plt.rcParams["xtick.labelsize"] = 10
plt.figure(figsize=(16,10)) # this creates a figure 16 inch wide, 10 inch high
from pprint import pprint
%matplotlib inline
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
# For modeling building and tunning
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
#from sklearn.preprocessing import Imputer
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [3]:
# for deep learning if I will have time

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from keras.utils.np_utils import to_categorical

Using TensorFlow backend.


In [4]:
# for evaluation

from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [5]:
from datetime import date
import datetime as dt

# 2)-Loading data

In [6]:
df = pd.read_csv('selected_feature.csv')
df.shape

(45805, 515)

In [7]:
df.head()

,event_type,distance,num_family,len_jour,ts_hour,origin_ADB,origin_ADL,origin_AER,origin_AGP,origin_AKL,...,dest_YEG,dest_YMQ,dest_YOW,dest_YTO,dest_YUL,dest_YVR,dest_YWG,dest_YYC,dest_YYZ,dest_ZRH
0,0,5834.154716,7,6.0,11,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,6525.926149,4,21.0,20,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,469.781624,2,3.0,23,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1498.817537,1,3.0,15,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,2921.339028,4,6.0,22,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 3)- Quick data check

### 3.5)- Target Variable

In [8]:
df_model=df.copy()

In [9]:
df_model.head()

,event_type,distance,num_family,len_jour,ts_hour,origin_ADB,origin_ADL,origin_AER,origin_AGP,origin_AKL,...,dest_YEG,dest_YMQ,dest_YOW,dest_YTO,dest_YUL,dest_YVR,dest_YWG,dest_YYC,dest_YYZ,dest_ZRH
0,0,5834.154716,7,6.0,11,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,6525.926149,4,21.0,20,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,469.781624,2,3.0,23,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1498.817537,1,3.0,15,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,2921.339028,4,6.0,22,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
target=df_model["event_type"]
features=df_model.drop(['event_type'], axis=1)

Remove extra variables such as origin, destination as we have got dummy variables

ts_datetime as we have got ts_hour

Also event_type as it is stored in target variable

In [11]:
print(target.shape)
print(features.shape)

(45805,)
(45805, 514)


In [12]:
features.head(2)

,distance,num_family,len_jour,ts_hour,origin_ADB,origin_ADL,origin_AER,origin_AGP,origin_AKL,origin_ALA,...,dest_YEG,dest_YMQ,dest_YOW,dest_YTO,dest_YUL,dest_YVR,dest_YWG,dest_YYC,dest_YYZ,dest_ZRH
0,5834.154716,7,6.0,11,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,6525.926149,4,21.0,20,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 3.6)-Normalize data

In [13]:
from sklearn.preprocessing import StandardScaler
X = StandardScaler().fit_transform(features)

# 4)-Model Building

In [14]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, target, test_size=0.3, random_state=0)

In [15]:
print(X_train.shape)
print(X_test.shape)

(32063, 514)
(13742, 514)


In [16]:
print(y_train.shape)
print(y_test.shape)

(32063,)
(13742,)


In [17]:
X_train

array([[-0.35177902,  1.0152115 , -0.61244991, ..., -0.03371257,
        -0.03135908, -0.08173844],
       [-0.50221793,  0.15374868, -0.15216897, ..., -0.03371257,
        -0.03135908, -0.08173844],
       [-0.69762774, -0.70771413, -0.21792339, ..., -0.03371257,
        -0.03135908, -0.08173844],
       ...,
       [ 1.98983893,  1.0152115 ,  3.53007847, ..., -0.03371257,
        -0.03135908, -0.08173844],
       [-0.50391064, -0.70771413,  0.76839289, ..., -0.03371257,
        -0.03135908, -0.08173844],
       [-0.73094009,  0.15374868, -0.34943223, ..., -0.03371257,
        -0.03135908, -0.08173844]])

In [18]:
# Logistic Classifeir
logreg = LogisticRegression(C=1e5)
logreg.fit(X_train, y_train)
predictions_LR = logreg.predict(X_test)

In [19]:
predictions_LR[:5]

array([0, 0, 0, 0, 0])

In [20]:
print(accuracy_score(y_test,predictions_LR))

0.9598311745015282


In [21]:
print(recall_score(y_test,predictions_LR))

0.0019047619047619048


In [22]:
print(classification_report(y_test,predictions_LR))

              precision    recall  f1-score   support

           0       0.96      1.00      0.98     13217
           1       0.03      0.00      0.00       525

    accuracy                           0.96     13742
   macro avg       0.50      0.50      0.49     13742
weighted avg       0.93      0.96      0.94     13742



# Solution1: Under-sampling method

In [23]:
booking = df_model[df_model['event_type']==1]

search = df_model[df_model['event_type']==0]

In [24]:
target=df_model["event_type"]
features=df_model.drop(['event_type'], axis=1)
X = StandardScaler().fit_transform(features)
X_train, X_test, y_train, y_test = train_test_split(X, target, test_size=0.3, random_state=0)

In [25]:
from imblearn.under_sampling import NearMiss
# Implementing Undersampling for Handling Imbalanced 
nm = NearMiss()
X_under,y_under=nm.fit_sample(X_train,y_train)

In [26]:
print(X_under.shape,y_under.shape)

(2566, 514) (2566,)


In [27]:
# checking classes
print(y_train.value_counts())

0    30780
1     1283
Name: event_type, dtype: int64


In [28]:
print(y_under.value_counts())

1    1283
0    1283
Name: event_type, dtype: int64


In [29]:
print(y_test.value_counts())

0    13217
1      525
Name: event_type, dtype: int64


In [30]:
y_test.shape

(13742,)

In [31]:
# Logistic Classifeir
logreg = LogisticRegression(C=1e5)
logreg.fit(X_under, y_under)
predictions_LR = logreg.predict(X_test)

In [32]:
print(accuracy_score(predictions_LR,y_test))

0.2592053558433998


In [33]:
print(classification_report(predictions_LR,y_test))

              precision    recall  f1-score   support

           0       0.24      0.95      0.39      3351
           1       0.70      0.04      0.07     10391

    accuracy                           0.26     13742
   macro avg       0.47      0.49      0.23     13742
weighted avg       0.59      0.26      0.15     13742



# Solution 2: Over-Sampling

In [34]:
from imblearn.combine import SMOTETomek

In [35]:
# Implementing Oversampling for Handling Imbalanced 
smk = SMOTETomek(random_state=42)
X_over,y_over=smk.fit_sample(X_train,y_train)

In [36]:
X_over.shape,y_over.shape

((60868, 514), (60868,))

In [41]:
X_test.shape, y_test.shape

((13742, 514), (13742,))

In [38]:
logreg = LogisticRegression(C=1e5)
logreg.fit(X_over, y_over)
predictions_LR = logreg.predict(X_test)

In [39]:
print(accuracy_score(predictions_LR,y_test))

0.5406782127783437


In [40]:
print(classification_report(predictions_LR,y_test))

              precision    recall  f1-score   support

           0       0.54      0.97      0.69      7339
           1       0.59      0.05      0.09      6403

    accuracy                           0.54     13742
   macro avg       0.56      0.51      0.39     13742
weighted avg       0.56      0.54      0.41     13742



# Solution 3: Using class_weight option as "balanced" 

In [42]:
from sklearn.utils import class_weight

In [43]:
class_weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(y_train),
                                                 y_train)

In [50]:
logreg = LogisticRegression(class_weight="balanced")

In [51]:
logreg.fit(X_train, y_train)
predictions_LR = logreg.predict(X_test)

In [52]:
print(accuracy_score(predictions_LR,y_test))

0.5813564255566875


In [53]:
print(classification_report(predictions_LR,y_test))

              precision    recall  f1-score   support

           0       0.58      0.97      0.73      7908
           1       0.58      0.05      0.10      5834

    accuracy                           0.58     13742
   macro avg       0.58      0.51      0.41     13742
weighted avg       0.58      0.58      0.46     13742



**END OF NOTEBOOK3**